In [ ]:
#Adapted from https://keras.io/zh/examples/cifar10_cnn_tfaugment2d/

numberOfHiddenLayers = 2  #default = 5, if 0 then useSVM=True
generateLargeNetworkUntrained = True
useSparsity = True
if(useSparsity):
  sparsityProbabilityOfConnection = 0.1 #1-sparsity
#addSkipLayers = False  #skip layers not supported by keras model.add definition format

if(generateLargeNetworkUntrained):
  generateNetworkUntrained = True
  largeNetworkRatio = 1
  generateLargeNetworkExpansion = False
  if(generateLargeNetworkExpansion):
    generateLargeNetworkRatioExponential = True
else:
  generateNetworkUntrained = False
  generateLargeNetworkRatio = False

def kernelInitializerWithSparsity(shape, dtype=None):
  initialisedWeights = tf.random.normal(shape, dtype=dtype) #change to glorot_uniform?
  sparsityMatrixMask = tf.random.uniform(shape, minval=0.0, maxval=1.0, dtype=tf.dtypes.float32)
  sparsityMatrixMask = tf.math.less(sparsityMatrixMask, sparsityProbabilityOfConnection)
  sparsityMatrixMask = tf.cast(sparsityMatrixMask, dtype=tf.dtypes.float32)
  initialisedWeights = tf.multiply(initialisedWeights, sparsityMatrixMask)
  return initialisedWeights
if(useSparsity):
  kernelInitializer = kernelInitializerWithSparsity
else:
  kernelInitializer = 'glorot_uniform'
  
def getLayerRatio(layerIndex):
  layerRatio = 1
  if(generateLargeNetworkUntrained):
    if(generateLargeNetworkExpansion):
      if(generateLargeNetworkRatioExponential):
        layerRatio = largeNetworkRatio**layerIndex
      else:
        layerRatio = largeNetworkRatio * layerIndex
    else:
      layerRatio = largeNetworkRatio
  else:
    layerRatio = 1
  return int(layerRatio)


In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Lambda, MaxPooling2D
from keras import backend as K
import os

if K.backend() != 'tensorflow':
    raise RuntimeError('This example can only run with the '
                       'TensorFlow backend, '
                       'because it requires TF-native augmentation APIs')

import tensorflow as tf


batch_size = 32
num_classes = 10
epochs = 100
num_predictions = 20
save_dir = '/tmp/saved_models'
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
print("input_shape = ", input_shape)

model = Sequential()
#model.add(Input(shape=input_shape))
if(numberOfHiddenLayers >= 1):
  layerRatio = getLayerRatio(1)
  model.add(Conv2D(32*layerRatio, (3, 3), kernel_initializer=kernelInitializer, padding='same'))
  model.add(Activation('relu'))
if(numberOfHiddenLayers >= 2):
  layerRatio = getLayerRatio(2)
  model.add(Conv2D(32*layerRatio, (3, 3), kernel_initializer=kernelInitializer))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), kernel_initializer=kernelInitializer))
  model.add(Dropout(0.25))

if(numberOfHiddenLayers >= 3):
  layerRatio = getLayerRatio(3)
  model.add(Conv2D(64*layerRatio, (3, 3), kernel_initializer=kernelInitializer, padding='same'))
  model.add(Activation('relu'))
if(numberOfHiddenLayers >= 4):
  layerRatio = getLayerRatio(4)
  model.add(Conv2D(64*layerRatio, (3, 3), kernel_initializer=kernelInitializer))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), kernel_initializer=kernelInitializer))
  model.add(Dropout(0.25))

if(numberOfHiddenLayers >= 5):
  layerRatio = getLayerRatio(5)
  model.add(Flatten())
  model.add(Dense(512*generateLargeNetworkRatio, kernel_initializer=kernelInitializer))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))

model.add(Flatten())  #flatten hLast if necessary (ie numberOfHiddenLayers <4)
if(generateLargeNetworkUntrained):
  model.add(Lambda(lambda x: tf.keras.backend.stop_gradient(x)))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

print(model.summary())

# Save model and weights
#if not os.path.isdir(save_dir):
#    os.makedirs(save_dir)
#model_path = os.path.join(save_dir, model_name)
#model.save(model_path)
#print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])